In [1]:
# Import required libraries
import pandas as pd
import duckdb
import os
from datetime import datetime

print("✅ Libraries imported successfully!")
print(f"📁 Current working directory: {os.getcwd()}")
print(f"📂 Files in data directory: {os.listdir('../data')}")
print("\n�� Ready to load data!")

✅ Libraries imported successfully!
📁 Current working directory: /home/jovyan/work
📂 Files in data directory: ['restaurant.duckdb', '.ipynb_checkpoints', 'SoakPrep.csv']

�� Ready to load data!


In [2]:
# Load the CSV data
csv_path = '../data/SoakPrep.csv'
df = pd.read_csv(csv_path)

print(f"📊 Data shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")
print("\n📝 First 5 rows:")
print(df.head())

# Clean the data - remove empty rows
df_clean = df.dropna(subset=['Timestamp'])
print(f"\n🧹 After cleaning, data shape: {df_clean.shape}")
print(f"�� Removed {len(df) - len(df_clean)} empty rows")

📊 Data shape: (214, 7)
📋 Columns: ['Timestamp', 'User', 'Date', 'Time', 'Shift', 'Chhola Soak (KG)', 'Dal Soak (KG)']

📝 First 5 rows:
            Timestamp   User       Date         Time      Shift  \
0  1/31/2025 14:05:07  Mudit  30/1/2025  11:30:00 pm      Night   
1  1/31/2025 14:05:33  Mudit  31/1/2025   1:00:00 pm  Afternoon   
2  1/31/2025 17:12:01  Mudit  31/1/2025   5:00:00 pm  Afternoon   
3  1/31/2025 20:10:59   Ayan  31/1/2025   9:00:00 am      Night   
4   2/1/2025 12:07:59   Ayan   1/2/2025          NaN  Afternoon   

   Chhola Soak (KG)  Dal Soak (KG)  
0              12.0            8.0  
1               4.0            0.0  
2               2.0            0.0  
3              12.0            6.0  
4               5.0            0.0  

🧹 After cleaning, data shape: (213, 7)
�� Removed 1 empty rows


In [3]:
# Connect to DuckDB and load data
db_path = '../data/restaurant.duckdb'
conn = duckdb.connect(db_path)

print(f"🔗 Connected to DuckDB: {db_path}")

# Create table from pandas DataFrame
conn.execute("DROP TABLE IF EXISTS soak_prep")
conn.execute("CREATE TABLE soak_prep AS SELECT * FROM df_clean")

print("✅ Data loaded into DuckDB successfully!")

# Verify data was loaded
result = conn.execute("SELECT COUNT(*) as total_records FROM soak_prep").fetchall()
print(f"📊 Total records in database: {result[0][0]}")

# Show table schema
print("\n📋 Table schema:")
schema = conn.execute("DESCRIBE soak_prep").fetchall()
for col in schema:
    print(f"  {col[0]}: {col[1]}")

🔗 Connected to DuckDB: ../data/restaurant.duckdb
✅ Data loaded into DuckDB successfully!
📊 Total records in database: 213

📋 Table schema:
  Timestamp: VARCHAR
  User: VARCHAR
  Date: VARCHAR
  Time: VARCHAR
  Shift: VARCHAR
  Chhola Soak (KG): DOUBLE
  Dal Soak (KG): DOUBLE


In [4]:
print("�� === BASIC ANALYSIS ===\n")

# 1. Total soak quantities by user
print("👥 1. Total soak quantities by user:")
user_totals = conn.execute("""
    SELECT 
        User,
        SUM("Chhola Soak (KG)") as total_chhola,
        SUM("Dal Soak (KG)") as total_dal,
        COUNT(*) as total_entries
    FROM soak_prep 
    GROUP BY User 
    ORDER BY total_chhola DESC
""").fetchall()

for row in user_totals:
    print(f"  {row[0]}: {row[1]}kg chhola, {row[2]}kg dal ({row[3]} entries)")

print("\n📅 2. Daily totals (last 10 days):")
daily_totals = conn.execute("""
    SELECT 
        Date,
        SUM("Chhola Soak (KG)") as daily_chhola,
        SUM("Dal Soak (KG)") as daily_dal
    FROM soak_prep 
    GROUP BY Date 
    ORDER BY Date DESC
    LIMIT 10
""").fetchall()

for row in daily_totals:
    print(f"  {row[0]}: {row[1]}kg chhola, {row[2]}kg dal")

�� === BASIC ANALYSIS ===

👥 1. Total soak quantities by user:
  Ayan: 1189.5kg chhola, 448.59999999999997kg dal (116 entries)
  Naresh : 448.7kg chhola, 116.5kg dal (36 entries)
  Munchun Kumar : 293.5kg chhola, 86.0kg dal (22 entries)
  Manchun kumar : 132.0kg chhola, 39.5kg dal (13 entries)
  Manchun kumar: 109.0kg chhola, 33.0kg dal (7 entries)
  Ayan : 41.0kg chhola, 19.0kg dal (4 entries)
  Akash: 34.0kg chhola, 12.0kg dal (4 entries)
  ayan: 28.0kg chhola, 12.0kg dal (2 entries)
  Manchu kumar: 26.0kg chhola, 8.0kg dal (2 entries)
  Mudit: 18.0kg chhola, 8.0kg dal (3 entries)
  Munchun Kumar: 14.0kg chhola, 5.0kg dal (1 entries)
  12: 12.0kg chhola, 5.0kg dal (1 entries)
  naresh: 9.5kg chhola, 3.5kg dal (1 entries)
  Naresh: 4.0kg chhola, 0.0kg dal (1 entries)

📅 2. Daily totals (last 10 days):
  9/7/2025: 13.0kg chhola, 4.0kg dal
  9/6/2025: 12.0kg chhola, 4.0kg dal
  9/4/2025: 14.0kg chhola, 5.0kg dal
  9/3/2025: 7.0kg chhola, 3.0kg dal
  9/2/2025: 12.0kg chhola, 6.0kg dal
  

In [5]:
print("�� === SHIFT ANALYSIS ===\n")

shift_analysis = conn.execute("""
    SELECT 
        Shift,
        COUNT(*) as entries,
        AVG("Chhola Soak (KG)") as avg_chhola,
        AVG("Dal Soak (KG)") as avg_dal,
        SUM("Chhola Soak (KG)") as total_chhola,
        SUM("Dal Soak (KG)") as total_dal
    FROM soak_prep 
    WHERE Shift IS NOT NULL
    GROUP BY Shift 
    ORDER BY total_chhola DESC
""").fetchall()

for row in shift_analysis:
    print(f"{row[0]} shift: {row[1]} entries")
    print(f"  Average: {row[2]:.1f}kg chhola, {row[3]:.1f}kg dal")
    print(f"  Total: {row[4]}kg chhola, {row[5]}kg dal\n")

�� === SHIFT ANALYSIS ===

Night shift: 159 entries
  Average: 13.5kg chhola, 4.9kg dal
  Total: 2140.5kg chhola, 784.0999999999999kg dal

Afternoon shift: 54 entries
  Average: 4.0kg chhola, 0.2kg dal
  Total: 218.7kg chhola, 12.0kg dal



In [7]:
# Close the connection
conn.close()
print("🔒 DuckDB connection closed.")

🔒 DuckDB connection closed.
